In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [4]:
# Install Pipeline SDK - This only needs to be ran once in the enviroment. 
# you can find the latest package @ https://github.com/kubeflow/pipelines/releases

#KFP_PACKAGE = 'https://storage.googleapis.com/ml-pipeline/release/0.1.21/kfp.tar.gz'
#!pip3 install $KFP_PACKAGE --upgrade

# Hello world with KubeFlow Pipelines 

Welcome to your first step with KubeFlow Pipelines(KFP). In this notebook, we will demo: 

* Defining a KubeFlow pipeline with Python KFP SDK
* Creating an experiment and submitting pipelines to KFP run time enviroment using the KFP SDK 

Reference documentation: 
* https://www.kubeflow.org/docs/pipelines/sdk/build-component/
* https://www.kubeflow.org/docs/pipelines/sdk/sdk-overview/

## Setup

In [21]:
# Set your output and project. !!!Must Do before you can proceed!!!
EXPERIMENT_NAME = 'Hellow world!'
PROJECT_NAME =  'chavoshi-dev-2'                      #'Your-Gcp-Project-Name'
OUTPUT_DIR = 'gs://chavoshi-dev-mlpipeline'          # A path for asset outputs
BASE_IMAGE='tensorflow/tensorflow:1.11.0-py3'         # Based image used in various steps of the pipeline
TARGET_IMAGE='gcr.io/%s/pusher:latest' % PROJECT_NAME # Target image that will include our final code

## Create an Experiment in the Pipeline System

Pipeline system requires an "Experiment" to group pipeline runs. You can create a new experiment, or call client.list_experiments() to get existing ones.

In [27]:
# Note that this notebook should be running in JupyterHub in the same cluster as the pipeline system.
# Otherwise it will fail to talk to the pipeline system.
import kfp
import kfp.dsl as dsl
from kfp.gcp import use_gcp_secret
from kubernetes import client as k8s_client
from kfp import compiler
from kfp import notebook
from kfp import components as comp

In [28]:
@dsl.python_component(
    name='add_op',
    description='adds two numbers',
    base_image=BASE_IMAGE  # note you can define the base image here, or during build time. 
)
def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    print(a)
    return a + b

### Build a Pipeline Step With the Above Function(Note: run either of the two options below)
#### Option One: Specify the dependency directly

In [33]:
from kfp import compiler

# The return value "DeployerOp" represents a step that can be used directly in a pipeline function
add_op = compiler.build_python_component(
    component_func=add,
    staging_gcs_path=OUTPUT_DIR,
    dependency=[kfp.compiler.VersionedDependency(name='google-api-python-client', version='1.7.0')],
    base_image=BASE_IMAGE,
    target_image=TARGET_IMAGE)

2019-06-20 14:02:12:INFO:Build an image that is based on tensorflow/tensorflow:1.11.0-py3 and push the image to gcr.io/chavoshi-dev-2/pusher:latest
2019-06-20 14:02:12:INFO:Checking path: gs://chavoshi-dev-mlpipeline...
2019-06-20 14:02:12:INFO:Generate entrypoint and serialization codes.
2019-06-20 14:02:12:INFO:Generate build files.
2019-06-20 14:02:13:INFO:Start a kaniko job for build.
2019-06-20 14:02:13:INFO:Cannot Find local kubernetes config. Trying in-cluster config.
2019-06-20 14:02:13:INFO:Initialized with in-cluster config.
2019-06-20 14:02:18:INFO:5 seconds: waiting for job to complete
2019-06-20 14:02:23:INFO:10 seconds: waiting for job to complete
2019-06-20 14:02:28:INFO:15 seconds: waiting for job to complete
2019-06-20 14:02:33:INFO:20 seconds: waiting for job to complete
2019-06-20 14:02:38:INFO:25 seconds: waiting for job to complete
2019-06-20 14:02:43:INFO:30 seconds: waiting for job to complete
2019-06-20 14:02:48:INFO:35 seconds: waiting for job to complete
2019-

#### Option Two: build a base docker container image with both tensorflow and google api client packages

In [29]:
%%docker {TARGET_IMAGE} {OUTPUT_DIR}
FROM {BASE_IMAGE}
RUN pip3 install google-api-python-client

2019-06-20 13:54:18:INFO:Checking path: gs://chavoshi-dev-mlpipeline...
2019-06-20 13:54:18:INFO:Generate build files.
2019-06-20 13:54:19:INFO:Start a kaniko job for build.
2019-06-20 13:54:19:INFO:Cannot Find local kubernetes config. Trying in-cluster config.
2019-06-20 13:54:19:INFO:Initialized with in-cluster config.
2019-06-20 13:54:24:INFO:5 seconds: waiting for job to complete
2019-06-20 13:54:29:INFO:10 seconds: waiting for job to complete
2019-06-20 13:54:29:INFO:Kubernetes job failed.
2019-06-20 13:54:29:INFO:Kaniko job complete.
2019-06-20 13:54:29:INFO:Build image complete.


Once the base docker container image is built, we can build a "target" container image that is base_image plus the python function as entry point. The target container image can be used as a step in a pipeline.

In [30]:
from kfp import compiler

# The return value "DeployerOp" represents a step that can be used directly in a pipeline function
add_op = compiler.build_python_component(
    component_func=add,
    staging_gcs_path=OUTPUT_DIR,
    target_image=TARGET_IMAGE)

2019-06-20 13:55:30:INFO:Build an image that is based on tensorflow/tensorflow:1.11.0-py3 and push the image to gcr.io/chavoshi-dev-2/pusher:latest
2019-06-20 13:55:30:INFO:Checking path: gs://chavoshi-dev-mlpipeline...
2019-06-20 13:55:30:INFO:Generate entrypoint and serialization codes.
2019-06-20 13:55:30:INFO:Generate build files.
2019-06-20 13:55:30:INFO:Start a kaniko job for build.
2019-06-20 13:55:30:INFO:Cannot Find local kubernetes config. Trying in-cluster config.
2019-06-20 13:55:30:INFO:Initialized with in-cluster config.
2019-06-20 13:55:35:INFO:5 seconds: waiting for job to complete
2019-06-20 13:55:40:INFO:10 seconds: waiting for job to complete
2019-06-20 13:55:45:INFO:15 seconds: waiting for job to complete
2019-06-20 13:55:50:INFO:20 seconds: waiting for job to complete
2019-06-20 13:55:55:INFO:25 seconds: waiting for job to complete
2019-06-20 13:56:00:INFO:30 seconds: waiting for job to complete
2019-06-20 13:56:06:INFO:35 seconds: waiting for job to complete
2019-

### Build a pipeline using this component

In [34]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a='a',
   b='7',
   c='17',
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #You can create explicit dependancy between the tasks using xyz_task.after(abc_task)
    add_2_task = add_op(a, b)
    
    add_3_task = add_op(add_task.output, add_2_task.output)

### Complie the pipeline

In [35]:
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Submit the pipeline for execution

In [36]:
#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

2019-06-20 14:09:05:INFO:Creating experiment Hellow world!.


In [37]:
#Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

#This link leads to the run information page. 
#Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working

/opt/conda/lib/python3.6/site-packages/kfp/_client.py:194: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f)
